In [ ]:
# Controls

# Monte Carlo Controls
GENERATE_MONTE_CARLO = False
SAVE_MONTE_CARLO = False

# Monte Carlo Parameters
N_TRIALS = 100

# Correlation Function Figure Parameters
CORR_FUNC_RES = 2000
CORR_FUNC_L_MAX = 30

# General Controls
NEW_L_MAX = 3
FOLDER_PATH = 'data/monte-carlo-trials/'
FIGURE_FOLDER_PATH = 'data/figures/'
MAP_FOLDER_PATH = ''

In [2]:
import numpy as np
import healpy as hp
import seaborn as sns
import warnings
import matplotlib
from matplotlib import pyplot as plt
from greatcirclelibrary import *
warnings.filterwarnings('ignore')
matplotlib.rcParams['figure.figsize'] = [13, 9]
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams.update({'font.size': 20})

In [3]:
map_paths = ['COM_CompMap_CMB-nilc_2048_R1.20.fits',    # 0 - NILC
             'COM_CompMap_CMB-smica_2048_R1.20.fits',   # 1 - SMICA
             'COM_CompMap_CMB-sevem_2048_R1.12.fits',   # 2 - SEVEM
             'COM_CompMap_CMB-commrul_2048_R1.00.fits'] # 3 - COMMRUL

map_paths = MAP_FOLDER_PATH + np.array(map_paths, dtype=np.object)
map_names = ['NILC', 'SMICA', 'SEVEM', 'COMMANDER']

In [4]:
alms_real_no_dipole_or_monopole = preprocess_maps(map_paths, NEW_L_MAX)

In [5]:
# calculate 100 preferred dipoles for each map, using 20,000 different great circles each time
# save their alms and variance of variances

In [ ]:
if GENERATE_MONTE_CARLO:
    monte_carlo_alms = np.zeros((N_TRIALS, alms_real_no_dipole_or_monopole.shape[0], alms_real_no_dipole_or_monopole.shape[1]), dtype='complex')
    monte_carlo_var_of_vars = np.zeros((N_TRIALS, alms_real_no_dipole_or_monopole.shape[0]))

    for i in range(N_TRIALS):
        gc_pix_temp = generate_gcs()
        monte_carlo_var_of_vars[i] = np.var(multi_gc_vars(gc_pix_temp, alms_real_no_dipole_or_monopole), axis=1, ddof=1)
    
    for i in range(N_TRIALS):
        gc_pix_temp = generate_gcs()
        monte_carlo_alms[i] = get_pref_versions(alms_real_no_dipole_or_monopole, gc_pix_temp)
    
    if SAVE_MONTE_CARLO:
        np.save(FOLDER_PATH + 'monte_carlo_alms.npy', monte_carlo_alms)
        np.save(FOLDER_PATH + 'monte_carlo_var_of_vars.npy', monte_carlo_var_of_vars)

else:
    monte_carlo_alms = np.load(FOLDER_PATH + 'monte_carlo_alms.npy')
    monte_carlo_var_of_vars = np.load(FOLDER_PATH + 'monte_carlo_var_of_vars.npy')


In [ ]:
# calculate mean preferred dipole amplitude and direction

In [ ]:
dipole_cls = np.zeros([N_TRIALS, alms_real_no_dipole_or_monopole.shape[0]])

for i in range(dipole_cls.shape[0]):
    for j in range(dipole_cls.shape[1]):
        dipole_cls[i][j] = hp.alm2cl(monte_carlo_alms[i][j])[1]

In [ ]:
mean_dipole_cls = np.mean(dipole_cls, axis=0)

In [ ]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [ ]:
dipole_dirs = np.zeros([N_TRIALS, 4, 3])

for i in range(dipole_dirs.shape[0]):
    for j in range(dipole_dirs.shape[1]):
        dipole_dirs[i][j] = unit_vector(hp.pixelfunc.fit_dipole(hp.sphtfunc.alm2map(monte_carlo_alms[i][j], NSIDE))[1])

avg_dipole_dir = np.mean(dipole_dirs, axis=0)

In [ ]:
ang_dist = np.zeros([N_TRIALS, 4])

for i in range(ang_dist.shape[0]):
    for j in range(ang_dist.shape[1]):
        ang_dist[i][j] = angle_between(avg_dipole_dir[j], dipole_dirs[i][j])

In [ ]:
180*np.max(ang_dist, axis=0)/np.pi

In [ ]:
dipole_angles = np.zeros([alms_real_no_dipole_or_monopole.shape[0], 2])

for i in prange(dipole_angles.shape[0]):
    dipole_angle = hp.pixelfunc.vec2ang(avg_dipole_dir[i])
    dipole_angles[i][0] = dipole_angle[0][0]
    dipole_angles[i][1] = dipole_angle[1][0]

In [ ]:
if GENERATE_MONTE_CARLO:
    monte_carlo_pref_alms = np.zeros_like(alms_real_no_dipole_or_monopole)
    monte_carlo_pref_var_of_vars = np.zeros((N_TRIALS, alms_real_no_dipole_or_monopole.shape[0]))
    
    for i in range(monte_carlo_pref_alms.shape[0]):
        pref_dipole_temp = np.zeros_like(alms_real_no_dipole_or_monopole[i])
        pref_dipole_temp[hp.Alm.getidx(NEW_L_MAX, 1, 0)] = np.sqrt(3*mean_dipole_cls[i])
        hp.rotate_alm(pref_dipole_temp, 0, dipole_angles[i][0], dipole_angles[i][1])
        monte_carlo_pref_alms[i] = alms_real_no_dipole_or_monopole[i] + pref_dipole_temp
    
    for i in range(N_TRIALS):
        gc_pix_temp = generate_gcs()
        monte_carlo_pref_var_of_vars[i] = np.var(multi_gc_vars(gc_pix_temp, monte_carlo_pref_alms), axis=1, ddof=1)
        
    if SAVE_MONTE_CARLO:
        np.save(FOLDER_PATH + 'monte_carlo_alms_pref.npy', monte_carlo_pref_alms)
        np.save(FOLDER_PATH + 'monte_carlo_var_of_vars_pref.npy', monte_carlo_pref_var_of_vars)
        
else:
    monte_carlo_pref_alms = np.load(FOLDER_PATH + 'monte_carlo_alms_pref.npy')
    monte_carlo_pref_var_of_vars = np.load(FOLDER_PATH + 'monte_carlo_var_of_vars_pref.npy')

In [ ]:
# correlation function uncertainty
# calculate the correlation function for each dipole
# and plot the sigma point by point for each set of sims

In [ ]:
my_maps_full = np.zeros((len(map_paths), hp.nside2npix(NSIDE)))
for i in range(len(map_paths)):
    my_maps_full[i] = hp.pixelfunc.ud_grade(hp.read_map(map_paths[i]), NSIDE)

In [ ]:
def correlation_function_graphs_stacked_uncertainty(res=CORR_FUNC_RES, L_max=CORR_FUNC_L_MAX):
    sns.set_style('whitegrid')
    cls_0 = np.zeros((alms_real_no_dipole_or_monopole.shape[0], L_max + 1))
    cls_1 = np.zeros((alms_real_no_dipole_or_monopole.shape[0], L_max + 1))
    cls_2 = np.zeros((N_TRIALS, alms_real_no_dipole_or_monopole.shape[0], L_max + 1))
    theta = np.linspace(0, np.pi, res)
    deg = np.degrees(theta)
    correlation_values_0 = np.zeros((4, res))
    correlation_values_1 = np.zeros((4, res))
    correlation_values_2 = np.zeros((N_TRIALS, 4, res))
    
    for i in range(cls_0.shape[0]):
        cls_0[i] = hp.sphtfunc.anafast(my_maps_full[i], lmax=30)
        cls_0[i][0] = 0
        cls_0[i][1] = 0
        
        correlation_values_0[i] = correlation_function(cls_0[i])(theta)
        
    for i in range(cls_1.shape[0]):
        cls_1[i] = hp.sphtfunc.anafast(my_maps_full[i], lmax=CORR_FUNC_L_MAX)
        cls_1[i][0] = 0
        cls_1[i][1] = mean_dipole_cls[i]
        
        correlation_values_1[i] = correlation_function(cls_1[i])(theta)
        
    for i in range(cls_2.shape[0]):
        for j in range(cls_2.shape[1]):
            cls_2[i][j] = np.copy(cls_0[j])
            cls_2[i][j][1] = dipole_cls[i][j]
            correlation_values_2[i][j] = correlation_function(cls_2[i][j])(theta)
            
    
    
    colors=sns.color_palette('Paired')
    
    fig = plt.figure()
    ax1 = fig.add_axes([0.1, 0.5, 0.8, 0.4], xlim=(0,180),
                   xticklabels=[], ylim=(-200, 200))
    ax2 = fig.add_axes([0.1, 0.1, 0.8, 0.4],
                   ylim=(-200, 200), xlim=(0,180))

    for i in range(4):
        my_sd=np.std(correlation_values_2[:, i, :], axis=0)
        ax2.fill_between(deg, correlation_values_1[i]-10 * my_sd, correlation_values_1[i]+10 * my_sd, zorder=0.001, color='grey', alpha=0.35)

    
    for i in range(4):
        ax1.plot(deg, correlation_values_0[i], color=colors[2*i+1], label=map_names[i], alpha=0.8)
        ax2.plot(deg, correlation_values_1[i], color=colors[2*i+1], label=map_names[i], zorder=100, alpha=0.8)


    ax1.axvline(90, color = 'grey', lw=1)
    ax2.axvline(90, color = 'grey', lw=1)
    ax1.axhline(0, color = 'grey', lw=1)
    ax2.axhline(0, color = 'grey', lw=1)
    ax1.axhline(-200, color = 'black', lw=2)
    ax1.set_yticklabels([None, -100, 0, 100, 200])
    ax2.set_yticklabels([-200, -100, 0, 100, None])
    ax1.set_xticks([0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180])
    ax2.set_xticks([0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180])
    ax1.set_xlabel(r'$\Theta^{\circ}$')
    ax1.set_ylabel(r'$C(\Theta)$ $(\mu K^2)$')
    ax2.set_xlabel(r'$\Theta^{\circ}$')
    ax2.set_ylabel(r'$C(\Theta)$ $(\mu K^2)$')
    ax2.legend(loc=3, prop={'size': 15})

    plt.savefig(FIGURE_FOLDER_PATH + 'corrfunc.png')
    plt.show()
    

In [ ]:
correlation_function_graphs_stacked_uncertainty()

In [ ]:
pref_alms = monte_carlo_pref_alms[3]
pref_rotation = get_pref_rot(pref_alms)

In [ ]:
rot_to_pref_coords(pref_alms)[hp.Alm.getidx(NEW_L_MAX, 3, 3)]

In [ ]:
r = R.from_euler('zyz', pref_rotation)

In [ ]:
r.apply(avg_dipole_dir[3])

In [ ]:
proj = hp.projector.MollweideProj()
ang = proj.xy2ang(proj.ang2xy(60, 110, lonlat=True))
axis_of_evil_vec = -1 * hp.pixelfunc.ang2vec(*ang)

In [ ]:
pref_vecs = []

for i in range(4):
    pref_vecs.append(r.apply(avg_dipole_dir[i]))
    
pref_vecs.append(r.apply(axis_of_evil_vec))

pref_angles = []

for i in range(5):
    current_vec = hp.pixelfunc.vec2ang(pref_vecs[i])
    pref_angles.append(np.array([current_vec[0][0], current_vec[1][0]]))

In [ ]:
vecs = []
for i in range(4):
    vecs.append(avg_dipole_dir[i])
    
vecs.append(axis_of_evil_vec)

angles = []
for i in range(5):
    current_vec = hp.pixelfunc.vec2ang(vecs[i])
    angles.append(np.array([current_vec[0][0], current_vec[1][0]]))

In [ ]:
my_colormap = 'viridis'

In [ ]:
# fig, axes = plt.subplots(nrows=2,ncols=2)

# my_max = max(hp.sphtfunc.alm2map(pref_alms, NSIDE))
# my_min = min(hp.sphtfunc.alm2map(pref_alms, NSIDE))

# plt.axes(axes[0][0])
# hp.mollview(hp.sphtfunc.alm2map(get_l(pref_alms, 1), NSIDE), title='Preferred Dipole of ' + map_names[3].title(), unit='μK', min=my_min, max=my_max, cmap=my_colormap, cbar=True, hold=True)
# hp.graticule()
# colors=sns.color_palette('Paired')
# for i in range(len(map_names)):
#     hp.projscatter(angles[i], label=map_names[i], color=colors[2*i + 1], marker='x')

# hp.projscatter(angles[4], color='deeppink', label='Axis of Evil', marker='*')
# # plt.legend()

# for i in range(1, 3):
#     plt.axes(axes[i//2][i % 2])
#     hp.mollview(hp.sphtfunc.alm2map(get_l(pref_alms, i + 1), NSIDE), title='$\ell = '+ str(i + 1) + '$ of ' + map_names[3].title(), unit='μK', min=my_min, cbar=True, max=my_max, cmap=my_colormap, hold=True)
#     hp.graticule()
    
# plt.axes(axes[1][1])
# hp.mollview(hp.sphtfunc.alm2map(pref_alms, NSIDE), title='$\ell \leq 3$ of ' + map_names[3].title(), unit='μK', min=my_min, cbar=True, max=my_max, cmap=my_colormap, hold=True)
# hp.graticule()

# plt.subplots_adjust(wspace=0.01,hspace=0.01)
# plt.savefig(FIGURE_FOLDER_PATH + 'dipole_figure_through_3_mollweide.png')

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)

my_max = max(hp.sphtfunc.alm2map(pref_alms, NSIDE))
my_min = min(hp.sphtfunc.alm2map(pref_alms, NSIDE))

plt.axes(axes[0][0])
hp.mollview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), 1), NSIDE), title='Preferred Dipole of ' + map_names[3].title(), unit='μK', min=my_min, max=my_max, cmap=my_colormap, cbar=True, hold=True)

colors=sns.color_palette('Paired')
colors_custom = [colors[1], colors[3], colors[5], colors[7], colors[9]]
for i in range(len(map_names)):
    hp.projscatter(pref_angles[i], label=map_names[i], color=colors_custom[i], marker='P', zorder=10, linewidth=0.75, edgecolor='black')

hp.projscatter(pref_angles[4], color='white', label='Axis of Evil', marker='P', alpha=1, zorder=10, linewidth=0.5, edgecolor='black')
plt.legend(loc=3, prop={'size': 12})

for i in range(1, 3):
    plt.axes(axes[i//2][i % 2])
    hp.mollview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), i + 1), NSIDE), title='$\ell = '+ str(i + 1) + '$ of ' + map_names[3].title(), unit='μK', min=my_min, cbar=True, max=my_max, cmap=my_colormap, hold=True)
    
plt.axes(axes[1][1])
hp.mollview(hp.sphtfunc.alm2map(rot_to_pref_coords(pref_alms), NSIDE), title='$\ell \leq 3$ of ' + map_names[3].title(), unit='μK', min=my_min, cbar=True, max=my_max, cmap=my_colormap, hold=True)
hp.graticule()

plt.subplots_adjust(wspace=0.01, hspace=0.01)
plt.savefig(FIGURE_FOLDER_PATH + 'mollweide.png')

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=3)

# my_max = max(hp.sphtfunc.alm2map(pref_alms, NSIDE))
# my_min = min(hp.sphtfunc.alm2map(pref_alms, NSIDE))
my_min=-50
my_max=50

plt.subplot(2, 3, 2)
hp.mollview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), 1), NSIDE), title='Preferred Dipole', unit='μK', min=my_min, max=my_max, cmap=my_colormap, cbar=False, hold=True)

plt.subplot(1, 3, 1)
hp.orthview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), 1), NSIDE), half_sky=True, rot = (0, 90, 0), title='Preferred Dipole', unit='μK', min=my_min, cbar=False, max=my_max, cmap=my_colormap, hold=True)

colors=sns.color_palette('Paired')
colors_custom = [colors[1], colors[3], colors[5], colors[7], colors[9]]
for i in range(len(map_names)):
    hp.projscatter(pref_angles[i], label=map_names[i], color=colors_custom[i], marker='D', linewidth=0.75, edgecolor='black')

hp.projscatter(pref_angles[4], color='white', label='Axis of Evil', marker='P', linewidth=0.5, edgecolor='black')

plt.legend(loc=3, prop={'size': 10})

plt.subplot(2, 3, 3)
hp.mollview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), 2), NSIDE), title='$\ell = 2$', unit='μK', min=my_min, cbar=False, max=my_max, cmap=my_colormap, hold=True)

plt.subplot(2, 3, 5)
hp.mollview(hp.sphtfunc.alm2map(get_l(rot_to_pref_coords(pref_alms), 3), NSIDE), title='$\ell = 3$', unit='μK', min=my_min, cbar=False, max=my_max, cmap=my_colormap, hold=True)
        
plt.subplot(2, 3, 6)
hp.mollview(hp.sphtfunc.alm2map(rot_to_pref_coords(pref_alms), NSIDE), title='$\ell \leq 3$', unit='μK', min=my_min, cbar=False, max=my_max, cmap=my_colormap, hold=True)
plt.subplots_adjust(wspace=0.001, hspace=0.001, bottom=0.8)
hp.graticule()

ax = plt.gca()        

# Get the images on an axis
im = ax.images        

# cax = plt.axes([0.95, 0.3, 0.025, 0.4])
# cbar = plt.colorbar(im[0], cax=cax, ticks=[-50, -25, 0, 25, 50])
# cbar.set_label(r'$\mu K$', rotation='horizontal', verticalalignment='center')
cax = plt.axes([0.2, 0.15, 0.6, 0.025])
cbar = plt.colorbar(im[0], cax=cax, ticks=[-50, -25, 0, 25, 50], orientation='horizontal')
cbar.set_label(r'$\mu K$', rotation='horizontal', horizontalalignment='center')

# fig.tight_layout()

plt.savefig(FIGURE_FOLDER_PATH + 'mollweide.png')